<a href="https://colab.research.google.com/github/Riccorl/emergency-situation-awareness/blob/master/wir_colab_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://www.dropbox.com/s/owl26mc9n0ruxop/data.zip
!unzip data.zip

In [0]:
!wget https://crisisnlp.qcri.org/data/lrec2016/crisisNLP_word2vec_model_v1.2.zip
!unzip crisisNLP_word2vec_model_v1.2.zip

In [0]:
!rm -r __MACOSX
!mkdir  output

In [0]:
import nltk
nltk.download('stopwords')

In [29]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

app.py

In [32]:
import gensim

# import config
# import evaluation
# import preprocess
# import train
# import utils


def process():
    print("Loading tweets...")
    features, labels = load_datasets(
        CRISIS_TRAIN_DIR, NORMAL_TRAIN_DIR
    )
    print("Clearing tweets...")
    features = clear_tweets(features)
    print(features[0])
    print("Loading pre-trained embeddings...")
    # load the w2v matrix with genism
    w2v = gensim.models.KeyedVectors.load_word2vec_format(
        CRISIS_PRE_TRAINED, binary=True
    )
    # build the vocab from the w2v model
    w2v_vocab = vocab_from_w2v(w2v)
    print("Word2Vec model vocab len:", len(w2v_vocab))
    # build vocab from the dataset
    data_vocab = build_vocab([features])
    # filter pretrained w2v with words from the dataset
    w2v = restrict_w2v(w2v, set(data_vocab.keys()))
    w2v_vocab = vocab_from_w2v(w2v)
    write_dictionary(TRAIN_VOCAB, w2v_vocab)
    print("Cleaned vocab len:", len(w2v_vocab))
    # idx2word = {v: k for k, v in vocab.items()}
    return features, labels, data_vocab, w2v, w2v_vocab


def main():
    features, labels, vocab, w2v, w2v_vocab = process()
    model = train_keras(features, labels, w2v=w2v, w2v_vocab=w2v_vocab, batch_size=2048)
    evaluation.evaluate(model)


main()


Loading tweets...
Number of crisis tweets: 270000
Number of non-crisis tweets: 420000
Clearing tweets...
['two', '#saudis', 'die', '#mers', 'virus', 'toll', 'nears', '100']
Loading pre-trained embeddings...


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


Word2Vec model vocab len: 2152856
Cleaned vocab len: 100640


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:72: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 300)         30192000  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               240600    
_________________________________________________________________
dense_4 (Dense)              (None, 50)                10050     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 51        
Total params: 30,442,701
Trainable params: 250,701
Non-trainable params: 30,192,000
_________________________________________________________________
Starting training...
Epoch 1/5
 23/151 [===>..........................] - ETA: 3:22 - loss: 0.4158 - acc: 0

KeyboardInterrupt: ignored

config.py

In [0]:
import pathlib

# directories
DATA_DIR = pathlib.Path("data") # .resolve().parent.parent / "data"
RES_DIR = pathlib.Path("resources") # .resolve().parent.parent / "resources"
OUTPUT_DIR = pathlib.Path("output") # RES_DIR / "output"
VOCABS_DIR = RES_DIR / "vocabs"

TRAIN_DIR = pathlib.Path("train")
CRISIS_TRAIN_DIR = TRAIN_DIR / "crisisnlp"
CRISISLEX_DIR = TRAIN_DIR / "crisislex"
NORMAL_TRAIN_DIR = TRAIN_DIR / "normal"
EVAL_DIR = pathlib.Path("evaluation") # DATA_DIR / "evaluation"
CRISIS_EVAL_DIR = EVAL_DIR / "crisisnlp"
NORMAL_EVAL_DIR = EVAL_DIR / "normal"

# embeddings
CRISIS_PRE_TRAINED = pathlib.Path("crisisNLP_word2vec_model") / "crisisNLP_word_vector.bin" # RES_DIR / "embeddings" / "crisisNLP_emb.bin"

# vocabs
TRAIN_VOCAB = pathlib.Path("train_vocab.txt") # VOCABS_DIR / "train_vocab.txt"

# crisis nlp
PAKISTAN_EQ_TWEETS = CRISIS_TRAIN_DIR / "2013_pakistan_eq.csv"
CALIFORNIA_EQ_TWEETS = CRISIS_TRAIN_DIR / "2014_california_eq.csv"
CHILE_EQ_TWEETS = CRISIS_TRAIN_DIR / "2014_chile_eq.csv"
NEPAL_EQ_TWEETS = CRISIS_TRAIN_DIR / "2015_nepal_eq_cf_labels.csv"
EBOLA_TWEETS = CRISIS_TRAIN_DIR / "2014_ebola_virus.csv"
ODILE_HR_TWEETS = CRISIS_TRAIN_DIR / "2014_hurricane_odile.csv"
HAGUPIT_HR_TWEETS = CRISIS_TRAIN_DIR / "2014_typhoon_hagupit_cf_labels.csv"
PAM_HR_TWEETS = CRISIS_TRAIN_DIR / "2015_cyclone_pam_cf_labels.csv"
MERS_TWEETS = CRISIS_TRAIN_DIR / "2014_mers_cf_labels.csv"
PAKISTAN_FL_TWEETS = CRISIS_TRAIN_DIR / "2014_pakistan_floods_cf_labels.csv"
INDIA_FL_TWEETS = CRISIS_TRAIN_DIR / "2014_india_floods.csv"


models.py

In [0]:
import gensim
import numpy as np
import tensorflow as tf
import tensorflow_hub as tf_hub
from tensorflow import keras
from tensorflow.python.keras import Model
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.layers import (
    Dense,
    Embedding,
    Bidirectional,
    LSTM,
    Input,
    Layer,
)


def build_model(
    layer: keras.layers = LSTM,
    hidden_size: int = 256,
    input_length: int = None,
    dropout: float = 0.2,
    recurrent_dropout: float = 0.1,
    learning_rate: float = 0.002,
    vocab_size: int = None,
    word2vec: gensim.models.word2vec.Word2Vec = None,
    embedding_size: int = 300,
    train_embeddings: bool = False,
) -> Model:

    input_layer = Input(shape=(input_length,))
    if word2vec:
        em = get_keras_embedding(word2vec, train_embeddings)(input_layer)
    else:
        em = Embedding(
            vocab_size, embedding_size, input_length=input_length, mask_zero=True
        )(input_layer)
    # em = ELMoEmbedding(idx2word=idx2word, output_mode="default", trainable=True)(input_layer)

    lstm1 = Bidirectional(
        layer(
            units=hidden_size,
            dropout=dropout,
            recurrent_dropout=recurrent_dropout,
            return_sequences=False,
        )
    )(em)
    # lstm2 = Bidirectional(
    #     layer(units=hidden_size, dropout=dropout, recurrent_dropout=recurrent_dropout)
    # )(lstm1)

    dense = Dense(50, activation="relu")(lstm1)
    output = Dense(1, activation="sigmoid")(dense)
    model = Model(inputs=input_layer, outputs=output)
    optimizer = keras.optimizers.Adam()
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["acc"])
    model.summary()
    return model


def get_keras_embedding(
    word2vec: gensim.models.word2vec.Word2Vec, trainable: bool = False
) -> Embedding:
    """
    Return a Tensorflow Keras 'Embedding' layer with weights set as the
    Word2Vec model's learned word embeddings.
    :param word2vec: gensim Word2Vec model
    :param trainable: if False, the weights are frozen and stopped from being updated.
                      If True, the weights can/will be further trained/updated.
    :return: a tf.keras.layers.Embedding layer.
    """
    weights = word2vec.wv.vectors
    # random vector for pad
    pad = np.random.rand(1, weights.shape[1])
    # mean vector for unknowns
    unk = np.mean(weights, axis=0, keepdims=True)
    weights = np.concatenate((pad, unk, weights))

    return Embedding(
        input_dim=weights.shape[0],
        output_dim=weights.shape[1],
        weights=[weights],
        mask_zero=True,
        trainable=trainable,
    )


class ELMoEmbedding(Layer):
    def __init__(self, idx2word, output_mode="default", trainable=True, **kwargs):
        assert output_mode in [
            "default",
            "word_emb",
            "lstm_outputs1",
            "lstm_outputs2",
            "elmo",
        ]
        assert trainable in [True, False]
        self.idx2word = idx2word
        self.output_mode = output_mode
        self.trainable = trainable
        self.max_length = None
        self.word_mapping = None
        self.lookup_table = None
        self.elmo_model = None
        self.embedding = None
        super(ELMoEmbedding, self).__init__(**kwargs)

    def build(self, input_shape):
        self.max_length = input_shape[1]
        self.word_mapping = [
            x[1] for x in sorted(self.idx2word.items(), key=lambda x: x[0])
        ]
        self.lookup_table = tf.contrib.lookup.index_to_string_table_from_tensor(
            self.word_mapping, default_value="<UNK>"
        )
        self.lookup_table.init.run(session=K.get_session())
        self.elmo_model = tf_hub.Module(
            "https://tfhub.dev/google/elmo/2", trainable=self.trainable
        )
        super(ELMoEmbedding, self).build(input_shape)

    def call(self, x, **kwargs):
        x = tf.cast(x, dtype=tf.int64)
        sequence_lengths = tf.cast(tf.count_nonzero(x, axis=1), dtype=tf.int32)
        strings = self.lookup_table.lookup(x)
        inputs = {"tokens": strings, "sequence_len": sequence_lengths}
        return self.elmo_model(inputs, signature="tokens", as_dict=True)[
            self.output_mode
        ]

    def compute_output_shape(self, input_shape):
        if self.output_mode == "default":
            return input_shape[0], 1024
        if self.output_mode == "word_emb":
            return input_shape[0], self.max_length, 512
        if self.output_mode == "lstm_outputs1":
            return input_shape[0], self.max_length, 1024
        if self.output_mode == "lstm_outputs2":
            return input_shape[0], self.max_length, 1024
        if self.output_mode == "elmo":
            return input_shape[0], self.max_length, 1024

    def get_config(self):
        config = {"idx2word": self.idx2word, "output_mode": self.output_mode}
        return list(items())


sequence.py

In [0]:
from typing import Dict

import numpy as np
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.utils import Sequence


class TextSequence(Sequence):
    def __init__(
        self, x_set, y_set, batch_size, vocab, max_len, num_classes: int = None
    ):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.vocab = vocab
        self.max_len = max_len

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.compute_x(
            self.x[idx * self.batch_size : (idx + 1) * self.batch_size],
            self.vocab,
            max_len=self.max_len,
        )
        batch_y = np.array(self.y[idx * self.batch_size : (idx + 1) * self.batch_size])

        return batch_x, batch_y

    def compute_x(
        self, features, vocab: Dict[str, int], max_len: int = 200, pad: bool = True
    ) -> np.ndarray:
        """
        Compute the features X.
        :param features: feature file.
        :param vocab: vocab.
        :param max_len: max len to pad.
        :param pad: If True pad the matrix, otherwise return the matrix not padded.
        :return: the feature vectors.
        """
        data = [
            [vocab[word] if word in vocab else vocab["<UNK>"] for word in l]
            for l in features
        ]
        if pad:
            return pad_sequences(
                data, truncating="post", padding="post", maxlen=max_len
            )
        else:
            return np.array(data)


preprocess.py

In [0]:
import re
import string
from typing import Dict, List, Set

import gensim
import numpy as np
from keras_preprocessing.text import maketrans
from nltk.corpus import stopwords
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import TweetTokenizer


def build_vocab(data: List[List[str]]) -> Dict[str, int]:
    """
    Compute the vocab from the bigrams
    :param data: data set files
    :return: Dictionary from bigram to int
    """
    vocab = {"<PAD>": 0, "<UNK>": 1}
    for dataset in data:
        for sentence in dataset:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = len(vocab)
    return vocab


def vocab_from_w2v(word2vec: gensim.models.word2vec.Word2Vec) -> Dict[str, int]:
    """
    :param word2vec: trained Gensim Word2Vec model
    :return: a dictionary from token to int
    """
    vocab = {"<PAD>": 0, "<UNK>": 1}
    for index, word in enumerate(word2vec.wv.index2word):
        vocab[word] = index + 2
    return vocab


def clear_tweets(tweets: List[str]) -> List:
    """
    Clear list of tweets.
    :param tweets: list of tweets.
    :return: cleared tweets.
    """
    # preprocessing stuffs
    stops = set(stopwords.words("english")) | set(string.punctuation)
    html_regex = re.compile(r"^https?:\/\/.*[\r\n]*")
    tokenizer = TweetTokenizer()
    return [_clear_tweet(tweet, tokenizer, stops, html_regex) for tweet in tweets]


def _clear_tweet(tweet: str, tokenizer, stops: Set, html_regex) -> List:
    """
    Clean the tweet in input.
    :param tweet: tweet to clean.
    :param stops: set of stop words and punctuation to remove.
    :return: tweet cleaned.
    """
    return [word for word in tokenizer.tokenize(tweet.lower()) if word not in stops and not html_regex.search(word)]


def compute_x(
    features, vocab: Dict[str, int], max_len: int = 200, pad: bool = True
) -> np.ndarray:
    """
    Compute the features X.
    :param features: feature file.
    :param vocab: vocab.
    :param max_len: max len to pad.
    :param pad: If True pad the matrix, otherwise return the matrix not padded.
    :return: the feature vectors.
    """
    data = [
        [vocab[word] if word in vocab else vocab["<UNK>"] for word in l]
        for l in features
    ]
    if pad:
        return pad_sequences(data, truncating="post", padding="post", maxlen=max_len)
    else:
        return np.array(data)


def batch_generator(
    features: List[str],
    labels: List[str],
    vocab: Dict[str, int],
    batch_size: int = 32,
    max_input_len: int = 0,
):
    """
    Generates batches of data from features and labels,
    use it with Keras model.
    :param features: list of unigrams feature
    :param labels: list of labels
    :param vocab: unigram vocab
    :param batch_size: size of the batch to yield
    :param n_classes: number of classes
    :param max_input_len: max len of the input
    :return: processed features and labels, in batches
    """

    while True:
        for start in range(0, len(features), batch_size):
            end = start + batch_size
            max_len = len(max(features[start:end], key=len))

            if max_input_len > 0:
                # truncate the sequence
                max_len = max_len if max_len < max_input_len else max_input_len

            X_batch = compute_x(features[start:end], vocab, max_len=max_len)
            y_batch = np.array(labels[start:end])
            yield X_batch, y_batch


utils.py

In [0]:
import csv
import os
import re
import zipfile
from typing import List, Dict, Tuple

import nltk
import numpy as np
from nltk.corpus import stopwords

# import config


def read_txt(filename: str) -> List[str]:
    """
    Read the dataset line by line.
    :param filename: file to read
    :return: a list of lines
    """
    with open(filename, encoding="utf8") as file:
        f = (line.strip() for line in file)
        return [line for line in f if line]


def write_txt(filename: str, lines: List[str]):
    """
    Writes a list of string in a file.
    :param filename: path where to save the file.
    :param lines: list of strings to serilize.
    :return:
    """
    with open(filename, "w", encoding="utf8") as file:
        file.writelines(line + "\n" for line in lines)


def read_dictionary(filename: str) -> Dict:
    """
    Open a dictionary from file, in the format key -> value
    :param filename: file to read.
    :return: a dictionary.
    """
    with open(filename) as file:
        return {k: v for k, *v in (l.split() for l in file)}


def write_dictionary(filename: str, dictionary: Dict):
    """
    Writes a dictionary as a file.
    :param filename: file where to save the dictionary.
    :param dictionary: dictionary to serialize.
    :return:
    """
    with open(filename, mode="w") as file:
        for k, v in dictionary.items():
            file.write(k + " " + str(v) + "\n")


def merge_txt_files(input_file: List[str], output_filename: str):
    """
    Merge the given text files.
    :param input_file: list of strings.
    :param output_filename: filename of the output file
    """
    with open(output_filename, "w", encoding="utf8") as out_file:
        out_file.writelines(line + "\n" for line in input_file)


def load_datasets(crisis_path, normal_path, limit: int = 30000) -> Tuple[List[str], List[int]]:
    """
    This method is used to handle all datasets path to read.
    :return: a list of tweets
    """
    # parse crisis tweets
    crisis_tweets = read_datasets(crisis_path, limit)
    crisis_tweets_label = [1] * len(crisis_tweets)
    print("Number of crisis tweets:", len(crisis_tweets))

    # parse non-crisis tweets
    normal_tweets = read_datasets(normal_path, limit)
    normal_tweets_label = [0] * len(normal_tweets)
    print("Number of non-crisis tweets:", len(normal_tweets))
    return crisis_tweets + normal_tweets, crisis_tweets_label + normal_tweets_label


def read_datasets(path, limit: int = 30000) -> List[str]:
    """
    Read crisis tweets from crisisnlp folder.
    :return: list of tweets.
    """
    tweets = []
    for file in path.glob("./*.txt"):
        tweets += read_txt(file)[:limit]
    return tweets


def read_crisilex() -> List[str]:
    """
    Read crisis tweets from crisislex folder.
    :return: list of tweets.
    """
    tweets = []
    for file in CRISISLEX_DIR.glob("./*.csv"):
        tweets += _read_crisislex_csv(file)
    return tweets


def read_normal() -> List[str]:
    """
    Read non-crisis tweets from normal folder.
    :return: list of tweets.
    """
    tweets = []
    for file in list(NORMAL_TRAIN_DIR.glob("./*.csv"))[:13]:
        tweets += _read_csv(file)
    return tweets


def _read_csv(filename) -> List[str]:
    """
    Extract tweet from csv file.
    :param filename: csv file.
    :return: a list of tweets.
    """
    with open(filename, encoding="latin1") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=",")
        next(csv_reader)
        return [row[-1] for row in csv_reader]


def _read_crisislex_csv(filename) -> List[str]:
    """
    Extract tweet from csv file.
    :param filename: csv file.
    :return: a list of tweets.
    """
    with open(filename, encoding="utf8") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=",")
        next(csv_reader)
        return [row[1] for row in csv_reader]


def split_csv(filename: str, n_split: int):
    """
    Split a large text file in smaller files.
    :param filename: file to split.
    :param n_split: number of parts to split.
    :return:
    """
    data = []
    with open(filename, encoding="latin1") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=",")
        next(csv_reader)
        for row in csv_reader:
            data.append(row[-1])

    batch = len(data) // n_split
    for i in range(0, len(data), batch):
        j = i + batch
        filename_batch = str(filename).split(".")[0] + "_" + str(n_split) + ".txt"
        print("Writing", filename_batch)
        write_txt(filename_batch, data[i:j])
        n_split -= 1


def split_txt(filename: str, n_split: int):
    """
    Split a large text file in smaller files.
    :param filename: file to split.
    :param n_split: number of parts to split.
    :return:
    """
    data = read_txt(filename)
    batch = len(data) // n_split
    for i in range(0, len(data), batch):
        j = i + batch
        filename_batch = str(filename).split(".")[0] + "_" + str(n_split) + ".txt"
        print("Writing", filename_batch)
        write_txt(filename_batch, data[i:j])
        n_split -= 1


def unzip_all(paths: list):
    """
    This method is used to unzip all files.
    :param paths: a list of path to unzip
    :return: None
    """
    for path in paths:
        for subdirs, dirs, files in os.walk(path):
            for file in files:
                _, file_ext = os.path.splitext(file)
                if file_ext == ".zip":
                    unzip(subdirs, file)


def unzip(main_folder: str, file: str):
    """
    This method is used to unzip a file
    :param main_folder: folder to extract
    :param file: file to unzip
    :return: None
    """
    zip_ref = zipfile.ZipFile(os.path.join(main_folder, file), "r")
    zip_ref.extractall(main_folder)
    zip_ref.close()


def clear_text(text: str) -> str:
    """
    This method remove from a string of text
    every special character (tags,hash-tag, number,
    url,etc).
    :param text: a string of text
    :return: a string (a text) without
    special character
    """

    return " ".join((" ".join(re.compile("[^a-zA-Z\d\s:]").split(text))).split())


def stop_words(all_language: bool = False) -> list:
    """
    This method is used to generate stop words.
    The default language is English but setting
    the boolean variable to true it generates the
    stop words for all language.
    :param all_language: a boolean variable used
    as flag for the languages.
    :return: a list containing the stop
    words.
    """
    nltk.download("stopwords", quiet=True)

    if all_language:
        return stopwords.words(stopwords.fileids())
    else:
        return stopwords.words("english")


def restrict_w2v(w2v, restricted_word_set):
    """
    Retrain from w2v model only words in the restricted word set.
    :param w2v:
    :param restricted_word_set:
    :return:
    """
    new_vectors = []
    new_vocab = {}
    new_index2entity = []
    new_vectors_norm = []

    for i in range(len(w2v.vocab)):
        word = w2v.index2entity[i]
        vec = w2v.vectors[i]
        vocab = w2v.vocab[word]
        vec_norm = w2v.vectors_norm[i] if w2v.vectors_norm else []
        if word in restricted_word_set:
            vocab.index = len(new_index2entity)
            new_index2entity.append(word)
            new_vocab[word] = vocab
            new_vectors.append(vec)
            if vec_norm:
                new_vectors_norm.append(vec_norm)

    w2v.vocab = new_vocab
    w2v.vectors = np.array(new_vectors)
    w2v.index2entity = np.array(new_index2entity)
    w2v.index2word = np.array(new_index2entity)
    if new_vectors_norm:
        w2v.vectors_norm = np.array(new_vectors_norm)
    return w2v


def clean_embeddings(path_input: str, path_output: str, size: int):
    """
    Clean embeddings by removing non lemma_synset vectors.
    :param path_input: path to original embeddings.
    :param path_output: path to cleaned embeddings.
    :param size:
    :return:
    """
    old_emb = read_txt(path_input)
    filtered = [vector for vector in old_emb if "_bn:" in vector]
    write_txt(path_output, [str(len(filtered)) + " " + str(size)] + filtered)


def timer(start: float, end: float) -> str:
    """
    Timer function. Compute execution time from strart to end (end - start).
    :param start: start time
    :param end: end time
    :return: end - start
    """
    hours, rem = divmod(end - start, 3600)
    minutes, seconds = divmod(rem, 60)
    return "{:0>2}:{:0>2}:{:05.2f}".format(int(hours), int(minutes), seconds)


evaluation.py

In [12]:
import gensim
import sklearn
from tqdm import tqdm

# import utils
# import preprocess
# import config
import tensorflow as tf


def classification_report(y_test, y_pred, target):
    """
    Compute a classification report using sklearn.
    :param y_test: true labels.
    :param y_pred: predicted labels.
    :param target: label names.
    :return: a classification report from sklearn.
    """
    return sklearn.metrics.classification_report(
        y_test, y_pred, target_names=target
    )


def evaluate(model):
    """
    Evaluate the model printing a classification report (acc, prec, recall and f1).
    :param model: the trained model.
    :return:
    """

    x_test, y_test = load_datasets(CRISIS_EVAL_DIR, NORMAL_EVAL_DIR, limit=1000)
    x_test = clear_tweets(x_test)
    vocab = read_dictionary(TRAIN_VOCAB)
    x_test = compute_x(x_test, vocab, max_len=100)[:, :, 0]
    y_pred = model.predict(x_test, batch_size=64)
    y_pred = [1 if y > 0.5 else 0 for y in y_pred]
    cr = classification_report(y_test, y_pred, ["normal", "crisis"])
    print("Classification report : \n", cr)


if __name__ == '__main__':
    model = tf.keras.models.load_model(str(OUTPUT_DIR / "model.h5"))
    evaluate(model)


OSError: ignored

train.py

In [0]:
import time

import tensorflow as tf
from sklearn.model_selection import train_test_split

# import config
# import models
# import utils
# from sequence import TextSequence


def train_keras(
    features,
    labels,
    w2v,
    w2v_vocab,
    epochs: int = 5,
    hidden_size: int = 100,
    batch_size: int = 2048,
):

    tweets_tr, tweets_dev, labels_tr, labels_dev = train_test_split(
        features, labels, test_size=0.10
    )

    train_gen = TextSequence(
        tweets_tr, labels_tr, vocab=w2v_vocab, batch_size=batch_size, max_len=100
    )

    dev_gen = TextSequence(
        tweets_dev, labels_dev, vocab=w2v_vocab, batch_size=batch_size, max_len=100
    )

    model = build_model(
        layer=tf.keras.layers.GRU,
        hidden_size=hidden_size,
        dropout=0.4,
        recurrent_dropout=0.2,
        vocab_size=len(w2v_vocab),
        word2vec=w2v,
    )

    print("Starting training...")
    start = time.time()
    history = model.fit_generator(
        train_gen,
        steps_per_epoch=len(tweets_tr) // batch_size,
        epochs=epochs,
        validation_data=dev_gen,
        validation_steps=len(tweets_dev) // batch_size,
        shuffle=True
        # callbacks=[es, cp],
    )
    end = time.time()
    print(timer(start, end))
    model.save(str(OUTPUT_DIR / "model.h5"))
    print("Training complete.")

    return model
